In [ ]:
import torch
from torch import nn
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from models import dataset, training_and_testing
from models.LEDNet.models import lednet
from metrics_and_losses import metrics
from utils import segmentation_labels, utils
import matplotlib.pyplot as plt
from palette_classification import color_processing
import torchsummary

In [ ]:
# configuration
n_examples = 5
model_name = 'lednet_ccncsa'
weights_path = "models/weights/"
dataset_path = "headsegmentation_dataset_ccncsa/"

In [ ]:
# defining transforms
tH, tW = 256, 256
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225] # from ImageNet
image_transform = T.Compose([T.Resize((tH, tW)), T.Normalize(mean, std)])
target_transform = T.Compose([T.Resize((tH, tW))])

# fetching dataset
n_classes = len(segmentation_labels.labels)
img_paths, label_paths = dataset.get_paths(dataset_path + 'training.xml')
X_train, X_test, Y_train, Y_test = train_test_split(img_paths, label_paths, test_size=0.20, random_state=99, shuffle=True)
train_dataset = dataset.MyDataset(X_train, Y_train, image_transform, target_transform)
test_dataset = dataset.MyDataset(X_test, Y_test, image_transform, target_transform)

In [7]:
# training hyperparameters
device = 'cpu'
batch_size = 64
n_epochs = 50

# model, loss, score function
model = lednet.LEDNet(num_classes=n_classes, output_size=(tH, tW))
loss_fn = nn.CrossEntropyLoss()
score_fn = metrics.batch_mIoU

# optimizer
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# printing model summary
torchsummary.summary(model=model, input_size=(3, tH, tW), batch_size=batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         MaxPool2d-1          [64, 3, 128, 128]               0
            Conv2d-2         [64, 29, 128, 128]             812
       BatchNorm2d-3         [64, 32, 128, 128]              64
              ReLU-4         [64, 32, 128, 128]               0
  DownsamplerBlock-5         [64, 32, 128, 128]               0
            Conv2d-6         [64, 16, 128, 128]             784
              ReLU-7         [64, 16, 128, 128]               0
            Conv2d-8         [64, 16, 128, 128]             784
       BatchNorm2d-9         [64, 16, 128, 128]              32
             ReLU-10         [64, 16, 128, 128]               0
           Conv2d-11         [64, 16, 128, 128]             784
             ReLU-12         [64, 16, 128, 128]               0
           Conv2d-13         [64, 16, 128, 128]             784
      BatchNorm2d-14         [64, 16, 1

In [ ]:
# training
results = training_and_testing.train_model(
    device, model, test_dataset, batch_size, n_epochs, score_fn, loss_fn, optimizer, lr_scheduler=None, evaluate=True, verbose=True)

In [ ]:
# plotting training results
training_and_testing.plot_training_results(results, plotsize=(20, 6))

In [ ]:
# resetting model, optimizer, learning rate scheduler
final_model = lednet.LEDNet(num_classes=n_classes)
final_optimizer = torch.optim.Adam(final_model.parameters(), lr=learning_rate)

# re-training model on entire training set and saving its weights
final_n_epochs = 25
training_and_testing.train_model(
    device, final_model, train_dataset, batch_size, final_n_epochs, score_fn, loss_fn, final_optimizer, verbose=True)
torch.save(final_model.state_dict(), weights_path + model_name + '.pth')

In [ ]:
# loading pretrained final model
try:
    final_model
except:
    final_model = lednet.LEDNet(num_classes=n_classes)
    final_model.load_state_dict(torch.load(weights_path + model_name + '.pth'))

# testing model on test dataset
test_score_fn = metrics.batch_IoU
label_names = list(segmentation_labels.labels.keys())
batch_IoU = training_and_testing.test_model(device, final_model, test_dataset, batch_size, test_score_fn)
batch_IoU_with_labels = { label: score for label, score in list(zip(label_names, batch_IoU.tolist())) }
batch_mIoU = batch_IoU.mean().item()
for label in batch_IoU_with_labels:
    print(f'batch_IoU_{label}: {batch_IoU_with_labels[label]}')
print(f'batch_mIoU={batch_mIoU}')

In [ ]:
# plotting some random examples from test dataset
random_images = torch.zeros((n_examples, 3, tH, tW))
random_targets = torch.zeros((n_examples, n_classes, tH, tW))
for i in range(n_examples):
    random_idx = torch.randint(high=len(test_dataset), size=(1,))
    random_image, random_target = test_dataset[random_idx]
    random_images[i] = random_image
    random_targets[i] = random_target
with torch.no_grad():
    final_model.eval()
    random_output = final_model(random_images)[0]
channels_max, _ = torch.max(random_output, axis=1)
random_predictions = (random_output == channels_max.unsqueeze(axis=1))
for i in range(n_examples):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.title('Ground Truth')
    plt.imshow(utils.from_DHW_to_HWD(color_processing.colorize_segmentation_masks(random_targets[i], segmentation_labels.labels)))
    plt.subplot(1, 2, 2)
    plt.title('Prediction')
    plt.imshow(utils.from_DHW_to_HWD(color_processing.colorize_segmentation_masks(random_predictions[i], segmentation_labels.labels)))